In [1]:
# import pandas as pd
#
# test_label = pd.read_csv("test_label.csv")

In [16]:
# import numpy as np
# from tqdm import tqdm
#
# test_label_metrix = []
# for i in tqdm(test_label['vector']):
#     test_label_metrix.append(eval(i))

100%|██████████| 25000/25000 [06:44<00:00, 61.73it/s]


In [20]:
# test_label_metrix_np = np.array(test_label_metrix)

In [23]:
# test_label_metrix_np.shape
# np.save("test_label_np.npy", test_label_metrix_np)

In [1]:
import numpy as np

test_label_metrix = np.load('test_label_np.npy')

In [2]:
print(test_label_metrix.shape)
test_label_metrix[0]

(25000, 4375)


array([-4.3794274, -4.7746954, -1.321032 , ..., -3.0502648, -3.1049044,
       -4.999597 ], dtype=float32)

In [17]:
import pandas as pd

test_query = pd.read_csv("data/test_query.csv", header=None)
test_query = test_query[0].map(lambda x: eval(x))
print(test_query[245])

['双手', '男人', '滑车', '滑板车', '亮堂', '帽子', '东西', '滑轮', '溜冰场', '滑板', '大厅', '木棒']


In [1]:
import json
category_map = {}
with open("category_map.json", "r") as f:
    category_map = json.load(f)

In [11]:
import  numpy as np

test_query = np.load("query_label_processed.npy", allow_pickle=True)

test_query_np = []
for i in test_query:
    arr = np.zeros([4375])
    for j in i:
        if j in category_map:
            arr[category_map[j]] = 1
    test_query_np.append(arr)

In [12]:
test_query_np = np.array(test_query_np)

In [14]:
test_query[0]

['双手', '草坪', '球杆', '上挥', '墨镜', '帽子', '男士', '男人', '草地']

In [15]:
np.save("query_label_processed_np.npy", test_query_np)

In [2]:
import numpy as np
query_query_np = np.load("query_label_np.npy")

In [5]:
idx = np.where(query_query_np[0] == 1)
idx

(array([  37,  176,  265, 1175, 1634, 3186, 3289, 3685, 3758], dtype=int64),)

In [6]:
import json
with open("category_map.json", "r") as f:
    cate = json.load(f)

r_cate = {cate[i]: i for i in cate}

for i in idx[0]:
    print(i, r_cate[i])

37 双手
176 草坪
265 帽子
1175 上挥
1634 球杆
3186 男人
3289 草地
3685 墨镜
3758 男士


In [8]:
# 计算余弦相似度
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm1 * norm2)
    return similarity

In [12]:
def label_filter(query, label_vec):
    query = np.where(query == 1, True, False)
    return np.sum(label_vec[query])

In [10]:
test_label = pd.read_csv("test_label.csv")

In [6]:
test_label

,Unnamed: 0,filename,vector
0,0,00052fbb-bc66-4981-a5b8-657cece65474.jpg,"[0.12004861, -0.76521045, 0.15136011, 0.099144..."
1,1,00056100-079c-4340-bf3b-fa0c39672388.jpg,"[0.92215645, -0.9495212, 0.14619908, -0.628865..."
2,2,00066881-c3d3-42e7-8574-f4837dfa8e96.jpg,"[0.59581876, 0.012878652, 1.0706635, 0.0139149..."
3,3,0008378f-7e9e-49d5-b54d-04a271647aa9.jpg,"[0.42760938, -0.6176873, -0.07277152, 0.072084..."
4,4,000908b5-6f73-4582-a79f-2de5e7094800.jpg,"[0.24775293, -0.3519494, -0.2148024, 0.1405869..."
...,...,...,...
24995,24995,fff58952-7513-4144-8ce9-78c1112ed6fd.jpg,"[0.20735091, -0.70538855, 0.1336636, 0.5180926..."
24996,24996,fff5cb0a-6e89-42bd-ae14-989cda7f97d6.jpg,"[0.8947249, -0.59305406, -0.5700437, 0.0099920..."
24997,24997,fff8541d-425f-4a15-b2a9-34d32d09bb0e.jpg,"[0.18543395, -0.51685876, 0.26987132, -0.01460..."
24998,24998,fffc55c6-063a-45b5-9edb-61438bf5f98d.jpg,"[0.9293775, -0.507025, 0.2716504, 0.5068123, -..."


In [11]:
from tqdm import tqdm

with open("submit_v3.csv", "w") as f:

    # 对于每个查询，计算相似度并选择top_k图片
    for query_vec in tqdm(query_query_np):
        similarities = np.array([cosine_similarity(query_vec, img_vec) for img_vec in test_label_metrix])
        top_k_indices = np.argsort(-similarities)[:5]  # 假设选择相似度最高的5张图片
        line = ','.join(list(test_label['filename'][top_k_indices])) + '\n'
        f.write(line)
        # break


  0%|          | 8/25000 [00:03<3:00:40,  2.31it/s]


KeyboardInterrupt: 

In [ ]:
def voc_ap(rec, prec, true_num):
    mrec = np.concatenate(([0.], rec, [1.]))
    mpre = np.concatenate(([0.], prec, [0.]))
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])
    i = np.where(mrec[1:] != mrec[:-1])[0]
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

def voc_mAP(imagessetfilelist, num, return_each=False):
    if isinstance(imagessetfilelist, str):
        imagessetfilelist = [imagessetfilelist]
    lines = []
    for imagessetfile in imagessetfilelist:
        with open(imagessetfile, 'r') as f:
            lines.extend(f.readlines())

    seg = np.array([x.strip().split(' ') for x in lines]).astype(float)
    gt_label = seg[:,num:].astype(np.int32)
    num_target = np.sum(gt_label, axis=1, keepdims = True)


    sample_num = len(gt_label)
    class_num = num
    tp = np.zeros(sample_num)
    fp = np.zeros(sample_num)
    aps = []

    for class_id in range(class_num):
        confidence = seg[:,class_id]
        sorted_ind = np.argsort(-confidence)
        sorted_scores = np.sort(-confidence)
        sorted_label = [gt_label[x][class_id] for x in sorted_ind]

        for i in range(sample_num):
            tp[i] = (sorted_label[i]>0)
            fp[i] = (sorted_label[i]<=0)
        true_num = 0
        true_num = sum(tp)
        fp = np.cumsum(fp)
        tp = np.cumsum(tp)
        rec = tp / float(true_num)
        prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
        ap = voc_ap(rec, prec, true_num)
        aps += [ap]

    np.set_printoptions(precision=6, suppress=True)
    aps = np.array(aps) * 100
    mAP = np.mean(aps)
    if return_each:
        return mAP, aps
    return mAP